<a href="https://www.kaggle.com/code/mikedelong/sentence-transformer-eda?scriptVersionId=155989067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=651b4ae64c6cf845222676b937c9a864dda12027c539aa0a1f8794681095912e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
with open(file='/kaggle/input/the-jungle-book/jungleb.txt', encoding='utf-8', mode='r') as input_fp:
    text = input_fp.read()
len(text)

292391

In [3]:
import pandas as pd
from nltk.tokenize import sent_tokenize

df = pd.DataFrame(data={'text': sent_tokenize(text=text.replace('\n', ' '), language='english')})
df['text'] = df['text'].apply(func=lambda x: ' '.join(x.split()))
df = df.drop_duplicates(ignore_index=True)
df['char count'] = df['text'].str.len()
df['word count'] = df['text'].apply(func=lambda x: len(x.split()))
df = df[df['word count'] > 1] # we're going to ignore single-token sentences
df['text40'] = df['text'].str[:40]
df = df.sample(frac=1.0, random_state=2023)
df.shape

(2483, 4)

We are going to leave all the Project Gutenberg additions to the text in place.

In [4]:
from plotly.express import scatter
scatter(data_frame=df.sort_values(ascending=True, by='word count').reset_index(), y=['word count', 'char count'] )

In [5]:
from arrow import now
from sentence_transformers import SentenceTransformer
MODEL = ['albert-base-v2', 'all-MiniLM-L6-v2', 'bert-base-uncased', 'distilbert-base-uncased', 'google/t5-v1_1-base', 
         'microsof/deberta-base', 'microsoft/MiniLM-L12-H384-uncased', 'microsoft/deberta-v3-base', 'microsoft/mpnet-base', 
         'nghuyong/ernie-2.0-en', 'nreimers/MiniLMv2-L6-H768-distilled-from-BERT-Large', 'nreimers/TinyBERT_L-6_H-768_v2', 
         'roberta-base', 't5-base'][1]
model_start = now()
model = SentenceTransformer(model_name_or_path=MODEL,)
model.max_seq_length = 256
print('model max sequence length: {}'.format(model.max_seq_length))
embeddings = model.encode(df['text'].values.tolist())
print('model time: {}'.format(now() - model_start))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

model max sequence length: 256


Batches:   0%|          | 0/78 [00:00<?, ?it/s]

model time: 0:00:31.792787


In [6]:
from sentence_transformers import util

similarity_start = now()
cos_sim = util.cos_sim(embeddings, embeddings)

# todo use a comprehension here
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    if (i > 0 and i % 200 == 0):
        print('{}: {}'.format(now(), i))
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

print('{}: calculated similarities.'.format(now()))
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print('similarity time: {}'.format(now() - similarity_start))

2023-12-21T20:34:15.971302+00:00: 200
2023-12-21T20:34:20.368667+00:00: 400
2023-12-21T20:34:23.996876+00:00: 600
2023-12-21T20:34:27.592725+00:00: 800
2023-12-21T20:34:30.937029+00:00: 1000
2023-12-21T20:34:32.788654+00:00: 1200
2023-12-21T20:34:34.434744+00:00: 1400
2023-12-21T20:34:37.318923+00:00: 1600
2023-12-21T20:34:38.323312+00:00: 1800
2023-12-21T20:34:39.089390+00:00: 2000
2023-12-21T20:34:39.581271+00:00: 2200
2023-12-21T20:34:39.820537+00:00: 2400
2023-12-21T20:34:39.845390+00:00: calculated similarities.
similarity time: 0:04:34.731990


We have removed duplicate sentences so we should see very few bogus high-similarity pairs. This first look at similarities is a kind of sniff test: models that do a better job at clusters below will tend to produce more imaginative high similarity pairs here.

In [7]:
for score, i, j in all_sentence_combinations[:10]:
    print('similarity: {:.4f} indices: {} {} left: {} right: {}'.format(cos_sim[i][j], i, j, df['text'].values[i][:40], df['text'].values[j][:40], ))

similarity: 1.0000 indices: 465 2001 left: You may copy it, give it away or re-use  right: You may copy it, give it away or re-use 
similarity: 0.9776 indices: 933 1293 left: _All things made he--Shiva the Preserver right: All things made he--Shiva the Preserver.
similarity: 0.9463 indices: 1189 2049 left: Little Toomai stared again and again. right: Little Toomai stared once more.
similarity: 0.9316 indices: 326 1591 left: For a young king cobra? right: For a young cobra?
similarity: 0.9082 indices: 131 1714 left: Section 2. right: Section 1.
similarity: 0.8983 indices: 1995 2246 left: Project Gutenberg™ eBooks are often crea right: Project Gutenberg eBooks may be modified
similarity: 0.8972 indices: 593 1988 left: Two Tails is a great coward. right: None the less, Two Tails is a great cowa
similarity: 0.8959 indices: 1990 2357 left: “A man’s cub. right: “I speak for the man’s cub.
similarity: 0.8932 indices: 804 882 left: There are a few things that you can do w right: There are a lo

In [8]:
embeddings.shape

(2483, 384)

In [9]:
from sklearn.manifold import TSNE
tsne_start = now()
tsne = TSNE(n_components=2, n_iter=10000, verbose=1, init='pca', random_state=2023)
df[['t0', 't1',]] = tsne.fit_transform(X=embeddings)
print('TSNE time: {}'.format(now() - tsne_start))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2483 samples in 0.001s...
[t-SNE] Computed neighbors for 2483 samples in 0.325s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2483
[t-SNE] Computed conditional probabilities for sample 2000 / 2483
[t-SNE] Computed conditional probabilities for sample 2483 / 2483
[t-SNE] Mean sigma: 0.293029
[t-SNE] KL divergence after 250 iterations with early exaggeration: 79.351685
[t-SNE] KL divergence after 10000 iterations: 1.818721
TSNE time: 0:02:15.187184


In [10]:
from plotly.express import scatter
scatter(data_frame=df, x='t0', y='t1', hover_name='text40', color=df['text'].apply(func=lambda x: 'gutenberg' not in x.lower()))

Obviously we have a distinct Gutenberg cluster that is isolated; we would expect this because the Project Gutenberg boilerplate is written in a different style than the rest of the text.

Let's use a vectorizer to pick out particularly important words and see if the embeddings capture their sentences as being similar.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def find_term(arg:str, terms: list):
    lower = arg.lower().split()
    for term in terms:
        if term in lower:
            return term
    return 'none'

vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words='english', max_df=0.5, min_df=0.001)
fit_result = vectorizer.fit_transform(raw_documents=df['text'].values)
word_weight_df = pd.DataFrame(data={'word': vectorizer.get_feature_names_out(), 'weight': fit_result.toarray().sum(axis=0),}).sort_values(ascending=False, by='weight')
top_terms = [word for word in  word_weight_df.head(n=20)['word'].values.tolist() if word not in {'head', 'come', 'time', 'thou', 'big', 'know', 'like', 'said', 'good', 'people'}]
top_terms = top_terms + ['elephant', 'cub', 'rikki-tikki', 'khan', 'akela', 'cobra', 'kala',  'kotick', 'teddy'] # let's add in some others
print('top terms: {}'.format(sorted(top_terms)))
plot_df = df.copy()
plot_df['term'] = plot_df['text'].apply(func=find_term, args=(top_terms, ))

top terms: ['akela', 'bagheera', 'baloo', 'cobra', 'cub', 'elephant', 'jungle', 'kala', 'khan', 'kotick', 'little', 'man', 'mowgli', 'nag', 'rikki', 'rikki-tikki', 'shere', 'teddy', 'tikki', 'wolf']


In [12]:
from plotly.colors import qualitative
scatter(data_frame=plot_df, x='t0', y='t1', hover_name='text40', color='term', color_discrete_sequence=qualitative.Light24, height=800)

We do have sentences that are easily understood as being part of clusters based on their nouns, particularly when these are proper nouns. In other cases we have sentences that are too undistinguished - being too short or having only pronouns for nouns - that this approach won't tell us much about them.